In [1]:
import pandas as pd
import numpy as np
from rank_bm25 import BM25Okapi
import re # Untuk pembersihan teks sederhana

In [2]:
# --- 1. Konfigurasi ---
# !!! PENTING: Pastikan path ini sudah benar sesuai lokasi file CSV Anda !!!
CSV_FILE_PATH = "D:/IT DEL/Semester 8 (FINAL)/STBI/Information-Retrieval-Books-search-engine-main/Information-Retrieval-Books-search-engine-main/data_books_updated.csv"

# Kolom yang akan digabungkan untuk menjadi 'dokumen' (sesuaikan dengan CSV Anda)
TEXT_COLUMNS = ['Book-Title', 'Book-Author', 'Publisher', 'description']
# Kolom yang akan ditampilkan saat hasil pencarian muncul
DISPLAY_COLUMNS = ['Book-Title', 'Book-Author', 'Publisher']

In [3]:
# --- 2. Pemuatan Data ---
def load_data(filepath):
    """
    Memuat data dari file CSV menggunakan pandas.

    Args:
        filepath (str): Path menuju file CSV.

    Returns:
        pandas.DataFrame: DataFrame yang berisi data buku, atau None jika gagal.
    """
    try:
        # Mencoba membaca CSV, mungkin perlu menyesuaikan index_col jika ada
        df = pd.read_csv(filepath)
        print(f"Berhasil memuat {len(df)} baris dari {filepath}")

        # Periksa apakah kolom 'Unnamed: 0' ada dan hapus jika perlu
        if 'Unnamed: 0' in df.columns:
            df = df.drop(columns=['Unnamed: 0'])
            print("Kolom 'Unnamed: 0' dihapus.")

        # Periksa keberadaan kolom teks dan tangani nilai yang hilang (NaN)
        for col in TEXT_COLUMNS:
            if col not in df.columns:
                print(f"KESALAHAN: Kolom '{col}' tidak ditemukan di CSV. Periksa nama kolom.")
                return None
            df[col] = df[col].fillna('').astype(str) # Ganti NaN dengan string kosong

        # Periksa keberadaan kolom display
        for col in DISPLAY_COLUMNS:
             if col not in df.columns:
                print(f"PERINGATAN: Kolom display '{col}' tidak ditemukan, tidak akan ditampilkan.")
                # Anda bisa memutuskan untuk menghentikan atau melanjutkan di sini

        return df
    except FileNotFoundError:
        print(f"KESALAHAN: File tidak ditemukan di '{filepath}'. Pastikan path sudah benar.")
        return None
    except Exception as e:
        print(f"KESALAHAN saat memuat CSV: {e}")
        return None

In [4]:
# --- 3. Pra-pemrosesan & Tokenisasi ---
def preprocess_text(text):
    """
    Membersihkan dan melakukan tokenisasi teks sederhana.
    """
    if not isinstance(text, str): # Pastikan input adalah string
        return []
    text = text.lower()
    text = re.sub(r'\W+', ' ', text) # Hapus karakter non-alphanumeric
    tokens = text.split()
    return tokens

def create_corpus_and_tokenize(df):
    """
    Membuat korpus dari DataFrame dan melakukan tokenisasi.

    Args:
        df (pandas.DataFrame): DataFrame buku.

    Returns:
        tuple: (list_dokumen_gabungan, list_dokumen_tertokenisasi)
    """
    corpus = []
    tokenized_corpus = []

    print("Membuat korpus dan melakukan tokenisasi...")
    for index, row in df.iterrows():
        # Gabungkan teks dari kolom yang ditentukan
        doc_text = " ".join([row[col] for col in TEXT_COLUMNS if col in df.columns])
        corpus.append(doc_text)
        tokenized_corpus.append(preprocess_text(doc_text))

    print(f"Korpus dibuat dengan {len(corpus)} dokumen.")
    return corpus, tokenized_corpus

In [5]:
# --- 4. Model BM25 ---
def build_bm25_model(tokenized_corpus):
    """
    Membangun model BM25 dari korpus yang ditokenisasi.
    """
    print("Membangun model BM25...")
    bm25 = BM25Okapi(tokenized_corpus)
    print("Model BM25 siap.")
    return bm25

In [6]:
# --- 5. Fungsi Pencarian ---
def search(query, bm25_model, df, k=5):
    """
    Mencari dokumen teratas untuk kueri menggunakan BM25.

    Args:
        query (str): Kueri input.
        bm25_model (BM25Okapi): Model BM25 yang sudah dilatih.
        df (pandas.DataFrame): DataFrame asli untuk mengambil detail.
        k (int): Jumlah dokumen teratas yang ingin diambil.

    Returns:
        list: Daftar dictionary yang berisi detail buku teratas.
    """
    tokenized_query = preprocess_text(query)
    doc_scores = bm25_model.get_scores(tokenized_query)

    # Dapatkan k dokumen teratas (indeks dan skor)
    # Gunakan argpartition untuk efisiensi pada data besar, lalu sort top k
    top_k_indices = np.argsort(doc_scores)[::-1][:k]

    results = []
    for idx in top_k_indices:
        if doc_scores[idx] > 0: # Hanya tampilkan jika skor > 0
            # Ambil data dari DataFrame menggunakan .iloc[idx]
            row_data = df.iloc[idx]
            result_info = {}
            for col in DISPLAY_COLUMNS:
                if col in row_data:
                    result_info[col] = row_data[col]
                else:
                    result_info[col] = 'N/A' # Jika kolom display tidak ada

            result_info['score'] = doc_scores[idx]
            result_info['doc_index'] = idx
            results.append(result_info)

    return results


In [7]:
# --- 6. Interaksi Pengguna ---
def interactive_search(bm25_model, df):
    """
    Memungkinkan pengguna memasukkan kueri dan melihat hasilnya.
    """
    print("\n--- Pencarian Buku Interaktif ---")
    print("Masukkan kueri Anda (atau ketik 'keluar' untuk berhenti):")

    while True:
        try:
            user_query = input("Kueri > ")
            if user_query.lower() == 'keluar':
                break

            if not user_query.strip():
                print("Kueri tidak boleh kosong.")
                continue

            top_5_results = search(user_query, bm25_model, df, k=5)

            print("\n--- 5 Buku Teratas ---")
            if not top_5_results:
                print("Tidak ada buku yang cocok ditemukan.")
            else:
                for i, book in enumerate(top_5_results):
                    print(f"{i + 1}. {book.get('Book-Title', 'N/A')}")
                    print(f"   Penulis   : {book.get('Book-Author', 'N/A')}")
                    print(f"   Penerbit  : {book.get('Publisher', 'N/A')}")
                    print(f"   Skor BM25 : {book['score']:.4f}")
                    # print(f"   (Indeks Dokumen: {book['doc_index']})") # Bisa diaktifkan jika perlu

            print("-" * 30)

        except Exception as e:
            print(f"Terjadi kesalahan saat mencari: {e}")


In [8]:
# --- Main ---
if __name__ == "__main__":
    # 1. Muat data
    books_df = load_data(CSV_FILE_PATH)

    if books_df is not None:
        # 2. Buat korpus dan tokenisasi
        corpus_list, tokenized_corpus_list = create_corpus_and_tokenize(books_df)

        # 3. Bangun model BM25
        bm25_model = build_bm25_model(tokenized_corpus_list)

        # 4. Mulai sesi pencarian interaktif
        interactive_search(bm25_model, books_df)
    else:
        print("Gagal memulai program karena data tidak dapat dimuat.")
        print("Silakan periksa path file CSV dan nama kolom di dalam kode.")

Berhasil memuat 58477 baris dari D:/IT DEL/Semester 8 (FINAL)/STBI/Information-Retrieval-Books-search-engine-main/Information-Retrieval-Books-search-engine-main/data_books_updated.csv
Kolom 'Unnamed: 0' dihapus.
Membuat korpus dan melakukan tokenisasi...
Korpus dibuat dengan 58477 dokumen.
Membangun model BM25...
Model BM25 siap.

--- Pencarian Buku Interaktif ---
Masukkan kueri Anda (atau ketik 'keluar' untuk berhenti):


Kueri >  Love



--- 5 Buku Teratas ---
1. Prem Raas: Gujarati Edition
   Penulis   : Ketki Shah, Zeel Shah (Goodreads Author) (Editor), Harsh Shah (Photographer)
   Penerbit  : Ketki Shah
   Skor BM25 : 3.4044
2. Works of Love
   Penulis   : Søren Kierkegaard, Howard Vincent Hong (Editor), Edna Hatlestad Hong ((Editor))
   Penerbit  : Princeton University Press
   Skor BM25 : 3.3376
3. The Truth About Love
   Penulis   : Patrick Roscoe
   Penerbit  : Key Porter Books
   Skor BM25 : 3.3212
4. Precious Gifts of Love
   Penulis   : C.J. Good
   Penerbit  : Createspace Independent Publishing Platform
   Skor BM25 : 3.2668
5. The Art of Loving
   Penulis   : Erich Fromm, Peter D. Kramer (Introduction), Lê Phương Anh (Translator), Rainer Funk (Afterword)
   Penerbit  : Harper Perennial Modern Classics
   Skor BM25 : 3.2441
------------------------------


KeyboardInterrupt: Interrupted by user